# **Initialisation**
Initialisation de la base de données spark

In [18]:
import os
import findspark

findspark.init()

import spark


In [19]:
import configparser

config = configparser.ConfigParser()

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local").appName("Import").getOrCreate()

# **Extraction**
Premier regard sur la base de données 

In [23]:
json_files = os.listdir("/home/marins/3A-ENSTA/Big data project/CVE_Analysis/src/")
print(json_files)

for json in json_files:
    df_temp = spark.read.option("multiline","true").json("/home/marins/3A-ENSTA/Big data project/CVE_Analysis/src/"+json)

print("Schema initial : ")
df_temp.printSchema()

df = df.select(explode(col("CVE_Items")))


['nvdcve-1.1-2022.json', 'nvdcve-1.1-2020.json', 'nvdcve-1.1-2018.json', 'nvdcve-1.1-2016.json', 'nvdcve-1.1-2021.json', 'nvdcve-1.1-2019.json', 'nvdcve-1.1-2017.json']


Schema initial : 
root
 |-- ASSIGNER: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- attackComplexity: string (nullable = true)
 |-- attackVector: string (nullable = true)
 |-- availabilityImpact: string (nullable = true)
 |-- baseScore: double (nullable = true)
 |-- baseSeverity: string (nullable = true)
 |-- confidentialityImpact: string (nullable = true)
 |-- integrityImpact: string (nullable = true)
 |-- privilegesRequired: string (nullable = true)
 |-- scope: string (nullable = true)
 |-- userInteraction: string (nullable = true)



AnalysisException: Column 'CVE_Items' does not exist. Did you mean one of the following? [ASSIGNER, ID, scope, baseScore, attackVector, baseSeverity, userInteraction, integrityImpact, attackComplexity, availabilityImpact, privilegesRequired, confidentialityImpact];
'Project [explode('CVE_Items) AS List()]
+- Project [ASSIGNER#16, ID#17, attackComplexity#19, attackVector#20, availabilityImpact#21, baseScore#22, baseSeverity#23, confidentialityImpact#24, integrityImpact#25, privilegesRequired#26, scope#27, userInteraction#28]
   +- Project [ASSIGNER#16, ID#17, attackComplexity#19, attackVector#20, availabilityImpact#21, baseScore#22, baseSeverity#23, confidentialityImpact#24, integrityImpact#25, privilegesRequired#26, scope#27, userInteraction#28, vectorString#29]
      +- Project [col#12.cve.CVE_data_meta.ASSIGNER AS ASSIGNER#16, col#12.cve.CVE_data_meta.ID AS ID#17, col#12.impact.baseMetricV3.cvssV3.attackComplexity AS attackComplexity#19, col#12.impact.baseMetricV3.cvssV3.attackVector

Extraction des données pertinents via select(). 

In [ ]:
df = df.select("col.cve.CVE_data_meta.*","col.impact.baseMetricV3.cvssV3.*").drop("version").drop("vectorString")

print("Columns : ",df.columns)
#df.printSchema()

Columns :  ['ASSIGNER', 'ID', 'attackComplexity', 'attackVector', 'availabilityImpact', 'baseScore', 'baseSeverity', 'confidentialityImpact', 'integrityImpact', 'privilegesRequired', 'scope', 'userInteraction']


Affichage des premières lignes et du nombre de samples à disposition pour se faire une idée.

In [ ]:
print("Number of available rows : ", df.count())
df.show(5)

Number of available rows :  10510


+--------------------+-------------+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|            ASSIGNER|           ID|attackComplexity|attackVector|availabilityImpact|baseScore|baseSeverity|confidentialityImpact|integrityImpact|privilegesRequired|    scope|userInteraction|
+--------------------+-------------+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|       cve@mitre.org|CVE-2016-0001|            null|        null|              null|     null|        null|                 null|           null|              null|     null|           null|
|secure@microsoft.com|CVE-2016-0002|            HIGH|     NETWORK|              HIGH|      7.5|        HIGH|                 HIGH|           HIGH|              NONE|UNCHANGED|       REQUIRED|
|secure@microsoft.com|CVE-2016-0003|    

Essai de la fonction tail pour récupérer des données. Pas idéal. La fonction "collect()" semble bien plus adaptée.

In [ ]:
print(df.tail(1)) 

[Row(ASSIGNER='cve@mitre.org', ID='CVE-2016-9998', attackComplexity='LOW', attackVector='NETWORK', availabilityImpact='NONE', baseScore=6.1, baseSeverity='MEDIUM', confidentialityImpact='LOW', integrityImpact='LOW', privilegesRequired='NONE', scope='CHANGED', userInteraction='REQUIRED')]


# Machine Learning Prediction
Tentative de prédiction du baseScore d'une CVE via les données cvssV3 ingurgitées.


In [ ]:
import pandas as pd
pandasDF = df.toPandas()
print(pandasDF.columns)

print(pandasDF.shape[0])

pandasDF = pandasDF[pandasDF['baseScore'].notna()]

print(pandasDF.shape[0])


pandasDF.head(3)

Index(['ASSIGNER', 'ID', 'attackComplexity', 'attackVector',
       'availabilityImpact', 'baseScore', 'baseSeverity',
       'confidentialityImpact', 'integrityImpact', 'privilegesRequired',
       'scope', 'userInteraction'],
      dtype='object')
10510
9028


,ASSIGNER,ID,attackComplexity,attackVector,availabilityImpact,baseScore,baseSeverity,confidentialityImpact,integrityImpact,privilegesRequired,scope,userInteraction
1,secure@microsoft.com,CVE-2016-0002,HIGH,NETWORK,HIGH,7.5,HIGH,HIGH,HIGH,NONE,UNCHANGED,REQUIRED
2,secure@microsoft.com,CVE-2016-0003,LOW,NETWORK,HIGH,9.6,CRITICAL,HIGH,HIGH,NONE,CHANGED,REQUIRED
4,secure@microsoft.com,CVE-2016-0005,LOW,NETWORK,NONE,4.3,MEDIUM,NONE,LOW,NONE,UNCHANGED,REQUIRED


In [ ]:
target_column = "baseScore"
target = pandasDF[target_column]

categorical_columns = ["attackComplexity",
                       "attackVector",
                       "availabilityImpact",
                       "confidentialityImpact",
                       "integrityImpact",
                       "privilegesRequired",
                       "scope",
                       "userInteraction"]

data = pandasDF[categorical_columns]


Affichage des types des différentes colonnes

In [ ]:
target.dtypes

dtype('float64')

In [ ]:
data.dtypes

attackComplexity         object
attackVector             object
availabilityImpact       object
confidentialityImpact    object
integrityImpact          object
privilegesRequired       object
scope                    object
userInteraction          object
dtype: object

In [ ]:
import numpy as np
#pd.set_option("display.max_rows", None)

target.value_counts().sort_index()

len(np.where(np.isnan(target))[0])

#pandasDF.loc[116]

0

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# Onehot ==> Linéaire
# Ordinal = Treebased

model = make_pipeline(OneHotEncoder(),LinearRegression())
model

Pipeline(steps=[('onehotencoder', OneHotEncoder()),
                ('linearregression', LinearRegression())])

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv = 10)


In [ ]:
cv_results["test_score"].mean()

0.9554594699294275